## Overview
Gradle handle building, testing and deployment of Java project based on configurations specified in the build script. The core components are highlighted in the diagram below:

<img src="images/gradle_core.png" wdth=700 height=auto />

**Dependency:** a resource required by the project. Most likely a library. A dependency can depend upon another dependency, called as *transitive dependency*.

**Repository:** a location where dependencies can be found (or resource published to)

**Task:** a basic unit of work, like compiling code or running tests.

### Project Structure
A project structure can look like either:

<img src="images/gradle_project_structure.png" wdth=900 height=auto />

**gradlew and gradlew.bat:** used to run various gradle commands.

**settings.gradle:** defines root project and all subprojects within it.

**build.gradle:** contains build configuration of the project (or subproject)

## Task
Tasks are smallest unit of work. To execute a task (named tasks):

```bash
$ ./gradlew tasks
```

To run a task (build) in a specific project:
```bash
$ ./gradlew :subprojectA:build
```

The output is not verbose, it doesn't list all the pre-requisite tasks. To enable verbose logging, create a file name *gradle.properties* and add a line `org.gradle.console=verbose`.

Tasks:
- need *input* like files, configuration properties or output from other tasks
- perform an *action*
- and generate an *output* like files often put in the build directory
- can have *dependencies* like tasks to run before and tasks to run after

In terminal tasks, have text next to it called *outcome labels*. Example:
```
> Task :compileJava
> Task :processResources UP-TO-DATE
> Task :classes
> Task :resolveMainClassName UP-TO-DATE
> Task :bootJar
> Task :jar UP-TO-DATE
> Task :assemble
> Task :compileTestJava UP-TO-DATE
> Task :processTestResources NO-SOURCE
> Task :testClasses UP-TO-DATE
```

No label means task executed successfully. Gradle supports incremental build support for deterministic tasks (same input, same output). Which means if the task was run previously and no change has been done since, the task will not be re-run. `UP-TO-DATE` label represents this.

We can change input configuration options related to tasks. As an example, to remove debug information in compiled code:

In [ ]:
// Task is of type JavaCompile and has options property
tasks.named('compileJava') {
    options.isDebug = false
}

Some tasks don't do anything other than trigger all its dependencies. Such tasks are called as *lifecycle tasks*. Example `build` and `check` tasks.

## Plugin
Provides additional functionality to Gradle build system. It can
- add new task like `compileJava`, `test`
- add new configuration like `implementation`, `runtimeOnly`
- add new DSL elements like `publishing {}`

Plugin can be:
- **core plugin:** this comes bundled with Gradle and doesn't require a version to be specified. Example `java`, `application`
- **community plugin:** developed and maintained by Gradle community. This requires a version. Example `org.springframework.boot`
- **local plugin:** created by us and available on the local machine

To apply a plugin:

In [ ]:
// build.gradle
plugins {
    id 'java'
    id 'org.springframework.boot' version '3.5.8'
}

An example of community plugin is taskinfo which provides a tree like representation of a task and its dependencies:

In [ ]:
plugins {
    id 'org.barfuin.gradle.taskinfo' version '2.2.1'
}

This plugin adds 3 tasks: `tiJson`, `tiOrder` and `tiTree`. To run `tiTree`, pass a task to it:
```bash
$ ./gradlew tiTree test
```

And it outputs:
```
:test                                 (org.gradle.api.tasks.testing.Test)
+--- :classes                         (org.gradle.api.DefaultTask)
|    +--- :compileJava                (org.gradle.api.tasks.compile.JavaCompile)
|    `--- :processResources           (org.gradle.language.jvm.tasks.ProcessResources)
+--- :compileJava                     (org.gradle.api.tasks.compile.JavaCompile)
+--- :compileTestJava                 (org.gradle.api.tasks.compile.JavaCompile)
|    +--- :classes                    (org.gradle.api.DefaultTask)
|    |    +--- :compileJava           (org.gradle.api.tasks.compile.JavaCompile)
|    |    `--- :processResources      (org.gradle.language.jvm.tasks.ProcessResources)
|    `--- :compileJava                (org.gradle.api.tasks.compile.JavaCompile)
`--- :testClasses                     (org.gradle.api.DefaultTask)
     +--- :compileTestJava            (org.gradle.api.tasks.compile.JavaCompile)
     |    +--- :classes               (org.gradle.api.DefaultTask)
     |    |    +--- :compileJava      (org.gradle.api.tasks.compile.JavaCompile)
     |    |    `--- :processResources (org.gradle.language.jvm.tasks.ProcessResources)
     |    `--- :compileJava           (org.gradle.api.tasks.compile.JavaCompile)
     `--- :processTestResources       (org.gradle.language.jvm.tasks.ProcessResources)       
```

## Dependency Management
A dependency is an external code that the project requires. A dependency can be:
- a module which can have multiple different versions and is downloaded from a *repository*
- another subproject in the same project. As an example, there can be a common subproject which is a dependency for other project
- a dependency can also be present as a file, though this is not recommended

Maven Central is the defacto repository in Gradle:

In [ ]:
repositories {
    mavenCentral()
}

A module dependecy can be configured using `dependencies`:

In [ ]:
dependencies {
    implementation("com.google.guava:guava:30.1.1-jre")
}

There are three parts here:
- `implementation`: is the configuration
- `com.google.guava:guava`: is module id which which is composed of group name and library name separated by colon
- `30.1.1-jre`: is the version

### Dependency Configuration
As the number of dependencies grow, it takes longer to compile the project. There are certain dependencies that are not required during compilation, they are required during runtime. Gradle provides us a way to separate runtime dependency from compile-time ones. And to separate the ones that are required for testing.
- `implementation`: required during source compilation
- `testImplementation`: required during test compilation
- `runtimeOnly`: required only during runtime
- `testRuntimeOnly`: required only during test execution

In [ ]:
dependencies {
     implementation('com.okta.spring:okta-spring-boot-starter:3.0.3')

     // JDBC driver
     runtimeOnly('org.postgresql:postgresql:42.7.3')

     // Logging
     implementation("org.slf4j:slf4j-api:2.0.7") // Slf4j for compilation
     runtimeOnly("ch.qos.logback:logback-classic:1.4.11") // Logback as runtime binding

     testImplementation("org.junit.jupiter:junit-jupiter-api:5.10.0")
     testRuntimeOnly("org.junit.jupiter:junit-jupiter-engine:5.10.0")
}

As stated earlier, dependencies can depend upon other dependencies called as *transitive dependency*. We can view the entire dependency tree using the `dependencies` task.

```bash
$ ./gradlew dependencies
```

The output contains list of `implementation`, `testImplementation`, etc dependencies as defined in the build configuration.
```
implementation - Implementation dependencies for the 'main' feature. (n)
+--- com.okta.spring:okta-spring-boot-starter:3.0.3 (n)
+--- org.slf4j:slf4j-api:2.0.7 (n)

testImplementation - Implementation only dependencies for source set 'test'. (n)
\--- org.junit.jupiter:junit-jupiter-api:5.10.0 (n)

runtimeOnly - Runtime-only dependencies for the 'main' feature. (n)
\--- ch.qos.logback:logback-classic:1.4.11 (n)

testRuntimeOnly - Runtime only dependencies for source set 'test'. (n)
\--- org.junit.jupiter:junit-jupiter-engine:5.10.0
```

But there are other entries in the output as well like `compileClasspath`, `runtimeClasspath`, `testCompileClasspath`, `testRuntimeClasspath`, etc. Where are these coming from? Take `compileClasspath` for example:
```
compileClasspath - Compile classpath for source set 'main'.
+--- org.projectlombok:lombok -> 1.18.42
+--- com.fasterxml.jackson.jaxrs:jackson-jaxrs-json-provider:2.18.1
     +--- com.fasterxml.jackson.jaxrs:jackson-jaxrs-base:2.18.1 -> 2.19.4
...
```

The tree lists the dependencies added in build configuration, alongwith transitive dependencies. Now take the case when we have specified two dependencies - *A* and *B* having versions *a* and *b*. *B* transitively depends upon dependency *A* version *a'*. Now which version of dependency *A* should be pulled in the project? The answer is whichever version is higher. The entry `com.fasterxml.jackson.jaxrs:jackson-jaxrs-base:2.18.1 -> 2.19.4` is an example of that.

There are two types of dependency configurations:
- bucket: direct dependencies are the ones we defined
- resolved: includes the direct dependencies and transitive dependencies

Tasks like `compileJava` use resolved dependencies. `dependencies` task shows a legend at the bottom:
```
(c) - A dependency constraint, not a dependency. The dependency affected by the constraint occurs elsewhere in the tree.
(*) - Indicates repeated occurrences of a transitive dependency subtree. Gradle expands transitive dependency subtrees only once per project; repeat occurrences only display the root of the subtree, followed by this annotation.                                                                           (n) - A dependency or dependency configuration that cannot be resolved.
```

`(n)` represents bucket dependency.

### API vs Implementation
Dependencies as discussed previously contribute to increase in compilation time. Furthermore, if those dependencies have multiple transitive dependencies, it adds up. Not all transitive dependencies though are required for compilation. If classes of transitive dependencies are used in classes of direct dependencies as public method parameters or return type (or as public variables) then users of the direct dependency would need to know about those classes of transitive dependency during compilation.

As an example, lets say our app depends upon a direct dependency that provides math operation:

In [ ]:
public class MyApp {
    public void calculation(long base, long exponent) {
        BigNumber number = MathUtil.power(base, exponent); // Use of transitive dependency class
        // ...
    }
}

// MathUtil is a direct dependency
public class MathUtil {
    public static BigNumber power(long base, long exponent) {
        // ...
    }
}

// BigNumber comes from a transitive dependency. // api

In the above case, we need transitive dependency during compilation. In other case, transitive dependency is not needed during compilation, only at runtime. An example of that:

In [ ]:
public class MyApp {
    public void calculation(long input) {
        boolean isPrime = MathUtil.isPrime(input); // MyApp doesn't see reference to BigNumber
        // ...
    }
}

// MathUtil is a direct dependency
public class MathUtil {
    public static boolean isPrime(long input) {
        BigNumber b;  // Transitive dependency used inside a method and doesn't leak out
        // ...
        return output;
    }
}

// BigNumber comes from a transitive dependency. // implementation

Authors of libraries can define which transitive dependencies are needed at compile time and which are needed only at runtime:
- `implementation`: dependency will not appear in downstream user compiled classpath. Only runtime.
- `api`: will apear in both compile and runtime classpath. Makes sense only for library projects (use Java library plugin).

API dependencies leak to downstream projects compiled classpath.

In the first example, Gradle build configuration of `MathUtil` library looks like:

In [ ]:
dependencies {
    api("org.example:bignumbers:1.2")

    // Changing to implementation will prevent MyApp from compiling
}

In the second example, Gradle build configuration of `MathUtil` library looks like:

In [ ]:
dependencies {
    implementation("org.example:bignumbers:1.2")
}

As a library author one can use `com.autonomousapps.dependency-analysis` plugin to verify if we have defined library dependency as `api` or `implementation` correctly.

### Dependency Version
When we specify a dependency as:

In [ ]:
dependencies {
    implementation("org.apache.commons:commons-lang3:3.15.0")
}

Gradle can resolve to a higher version than 3.15.0. This can happen if for example there is a transitive dependency which requires a higher version like 3.16.0 causing a version conflict. Gradle resolves version conflict by choosing the higher version. Two different versions of the same library cannot exist.

If we want strict version, then:

In [ ]:
dependencies {
    implementation("org.apache.commons:commons-lang3:3.15.0!!")  // double exclamation

    // Another way
    implementation("org.apache.commons:commons-lang3") {
        version {
            strictly("3.15.0")
        }
    }
}

What happens if a transitive dependency has a different version? The build will fail with version conflict error. However, if the library specified the transitive dependencies version as a range, then the build will succeed if 3.15.0 lies in that range.

We can also specify version with wildcards like:

In [ ]:
implementation("org.apache.commons:commons-lang3:3.+") // highest version starting with 3.

When a version is upgraded, the output of `dependencies` task reveals that:
```
 \--- org.junit.jupiter:junit-jupiter-api:5.11.4 -> 5.12.2 (*)
```